**Run the Cell to import the packages**

In [ ]:
import pandas as pd
import numpy as np
import csv

**Fill in the Command to load your CSV dataset "imdb.csv" with pandas**

In [ ]:
#Data Loading
imdb=pd.read_csv(  "imdb.csv"   )
imdb.columns = ["index","text","label"]
print(imdb.head(5))

   index                                               text  label
0      0  A very, very, very slow-moving, aimless movie ...      0
1      1  Not sure who was more lost - the flat characte...      0
2      2  Attempting artiness with black & white and cle...      0
3      3       Very little music or anything to speak of.        0
4      4  The best scene in the movie was when Gerardo i...      1


**Data Analysis**

- Get the shape of the dataset and print it.

- Get the column names in list and print it.

- Group the dataset by **label** and describe the dataset to understand the basic statistics of the dataset.

- Print the first three rows of the dataset

In [ ]:
data_size = imdb.size

print(data_size)

imdb_col_names = imdb.columns

print(imdb_col_names)


3000
Index([u'index', u'text', u'label'], dtype='object')
()
()


**Target Identification**

Execute the below cell to identify the target variables. If 0 it is a bad review,if it is 1 it is a good review.


In [ ]:
imdb_target=imdb['label'] 

print(imdb_target)


0      0
1      0
2      0
3      0
4      1
5      0
6      0
7      1
8      0
9      1
10     1
11     1
12     1
13     1
14     1
15     0
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     0
26     0
27     1
28     1
29     1
      ..
970    1
971    1
972    0
973    0
974    0
975    1
976    1
977    0
978    1
979    1
980    1
981    1
982    1
983    1
984    1
985    1
986    1
987    1
988    1
989    1
990    1
991    1
992    1
993    1
994    0
995    0
996    0
997    0
998    0
999    0
Name: label, Length: 1000, dtype: int64


**Tokenization**

- Convert the text into lower.
- Tokenize the text using word_tokenize
- Apply the function **split_tokens** for the column **text** in the **imdb** dataset with axis =1

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('all')


def split_tokens(text):

  message = text.lower()
  message = unicode(message, 'utf8') #convert bytes into proper unicode 
  word_tokens = word_tokenize(message)

  return word_tokens

imdb['tokenized_message'] = imdb.apply(lambda row: split_tokens(row['text']),axis=1)

**Lemmatization**

- Apply the function **split_into_lemmas** for the column **tokenized_message** with axis=1
- Print the 55th row from the column **tokenized_message**.
- Print the 55th row from the column **lemmatized_message**

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

def split_into_lemmas(text):

    lemma = []

    lemmatizer = WordNetLemmatizer()

    for word in text:

        a=lemmatizer.lemmatize(word)
        
        lemma.append(a)

    return lemma

 

imdb['lemmatized_message'] = imdb.apply(lambda row: split_into_lemmas(row['tokenized_message']),axis=1)



print('Tokenized message:', imdb['tokenized_message'][55]                       )

print('Lemmatized message:', imdb['tokenized_message'][55]                     )

('Tokenized message:', [u'but', u'i', u'recommend', u'waiting', u'for', u'their', u'future', u'efforts', u',', u'let', u'this', u'one', u'go', u'.'])
('Lemmatized message:', [u'but', u'i', u'recommend', u'waiting', u'for', u'their', u'future', u'efforts', u',', u'let', u'this', u'one', u'go', u'.'])


**Stop Word Removal**
- Set the stop words language as english in the variable **stop_words**
- Apply the function **stopword_removal** to the column **lemmatized_message** with axis=1
- Print the 55th row from the column **preprocessed_message**

In [ ]:
from nltk.corpus import stopwords



def stopword_removal(text):

    stop_words = set(stopwords.words('english'))

    filtered_sentence = []

    filtered_sentence = ' '.join([word for word in text if word not in stop_words])

    return filtered_sentence



imdb['preprocessed_message'] = imdb.apply(lambda row: stopword_removal(row['lemmatized_message']),axis=1)

print('Preprocessed message:',imdb['preprocessed_message'][55])

Training_data=pd.Series(list(imdb['preprocessed_message']))

Training_label=pd.Series(list(imdb['label']))


('Preprocessed message:', u'recommend waiting future effort , let one go .')


**Term Document Matrix**

- Apply CountVectorizer with following parameters
  - ngram_range = (1,2)
  - min_df = (1/len(Training_label))
  - max_df = 0.7
- Fit the **tf_vectorizer** with the **Training_data**
- Transform the **Total_Dictionary_TDM** with the **Training_data** 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

tf_vectorizer = CountVectorizer(  ngram_range=(1, 2),min_df = (1/len(Training_label)), max_df = 0.7 )

Total_Dictionary_TDM = tf_vectorizer.fit(Training_data)

message_data_TDM = Total_Dictionary_TDM.transform(Training_data)


**Term Frequency Inverse Document Frequency (TFIDF)**
- Apply TfidfVectorizer with following parameters
  - ngram_range = (1,2)
  - min_df = (1/len(Training_label))
  - max_df = 0.7
- Fit the **tfidf_vectorizer** with the **Training_data**
- Transform the **Total_Dictionary_TFIDF** with the **Training_data** 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),min_df = (1/len(Training_label)), max_df = 0.7   )

Total_Dictionary_TFIDF = tfidf_vectorizer.fit(Training_data)

message_data_TFIDF = Total_Dictionary_TFIDF.transform(Training_data)


**Train and Test Data**

Splitting the data for training and testing(90% train,10% test)

- Perform train-test split on **message_data_TDM** and **Training_label** with 90% as train data and 10% as test      data.

In [ ]:
from sklearn.model_selection import train_test_split#Splitting the data for training and testing

train_data,test_data, train_label, test_label = train_test_split(message_data_TDM, Training_label, test_size=0.1)

**Support Vector Machine**

- Get the shape of the train-data and print the same.

- Get the shape of the test-data and print the same.

- Initialize SVM classifier with following parameters
    - kernel = linear
    - C= 0.025
    - random_state=seed

- Train the model with train_data and train_label

- Now predict the output with test_data

- Evaluate the classifier with score from test_data and test_label

- Print the predicted score

In [ ]:
seed=9
from sklearn.svm import SVC

train_data_shape = train_data.shape

test_data_shape = test_data.shape
print("The shape of train data", train_data_shape            )

print("The shape of test data", test_data_shape            )

classifier = SVC( kernel="linear", C=0.025,random_state=seed )

classifier = classifier.fit(train_data, train_label)

target =  classifier.predict(test_data)

score =  classifier.score(test_data, test_label)

print('SVM Classifier : ',score)


with open('output.txt', 'w') as file:
    file.write(str((imdb['tokenized_message'][55],imdb['lemmatized_message'][55])))

('The shape of train data', (900, 9054))
('The shape of test data', (100, 9054))
('SVM Classifier : ', 0.77)


**Stochastic Gradient Descent Classifier**

- Perform train-test split on **message_data_TDM** and **Training_label** with this time 80% as train data and     20% as test data.

- Get the shape of the train-data and print the same.

- Get the shape of the test-data and print the same.

- Initialize SVM classifier with following parameters
    - loss = modified_huber
    - shuffle= True
    - random_state=seed

- Train the model with train_data and train_label

- Now predict the output with test_data

- Evaluate the classifier with score from test_data and test_label

- Print the predicted score

In [ ]:
from sklearn.linear_model import SGDClassifier

train_data,test_data, train_label, test_label = train_test_split(message_data_TDM, Training_label, test_size=0.2)

train_data_shape = train_data.shape

test_data_shape = test_data.shape

print("The shape of train data", train_data_shape           )

print("The shape of test data", test_data_shape          )

classifier =  SGDClassifier(loss='modified_huber', shuffle=True,random_state=seed)

classifier = classifier.fit(train_data, train_label)

target=classifier.predict(test_data)

score = classifier.score(test_data, test_label)

print('SGD classifier : ',score)

with open('output1.txt', 'w') as file:
    file.write(str((imdb['preprocessed_message'][55])))

('The shape of train data', (800, 9054))
('The shape of test data', (200, 9054))
('SGD classifier : ', 0.795)


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
